In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn as nn
from sklearn.model_selection import KFold



In [2]:
# train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp32.csv' ,delimiter=',')
# test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp32.csv' ,delimiter=',')

train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp2.csv' ,delimiter=',')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp2.csv' ,delimiter=',')





In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



cuda


In [9]:
# Separate features and target
X = train.drop(columns=['price'])
y = ((train['price'])) # talk about this 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
# # Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# # Reshape y tensors to have the correct shape (n_samples, 1)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


In [12]:

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

def rmsle_loss(y_pred, y_true):
    # Add a small constant to avoid taking log of zero
    epsilon = 1e-6
    y_pred = torch.clamp(y_pred, min=epsilon, max=1e9)  # Clamp predictions to avoid log(0)
    y_true = torch.clamp(y_true, min=epsilon, max=1e9)
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))


In [13]:
from torch.optim.lr_scheduler import CyclicLR

def train_model(model, train_loader, test_loader, num_epochs=5):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-6, momentum=0.8)
    scheduler = CyclicLR(optimizer, base_lr=1e-6, max_lr=0.1, step_size_up=5*len(train_loader),
                     mode='triangular', cycle_momentum=False)


    min_train_rmsle = float('inf')
    min_test_rmsle = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = rmsle_loss(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update learning rate at each batch

            total_train_loss += loss.item()

        avg_train_rmsle = total_train_loss / len(train_loader)

        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                test_loss = rmsle_loss(outputs, targets)
                total_test_loss += test_loss.item()

        avg_test_rmsle = total_test_loss / len(test_loader)

        if avg_train_rmsle < min_train_rmsle:
            min_train_rmsle = avg_train_rmsle
        if avg_test_rmsle < min_test_rmsle:
            min_test_rmsle = avg_test_rmsle

        print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle} , Current LR = {scheduler.get_last_lr()}')

    print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
    print(f'Minimum Test RMSLE so far: {min_test_rmsle}')


In [10]:
# def train_model(model, train_loader, test_loader, num_epochs=50):
#     model.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

#     for epoch in range(num_epochs):
#         model.train()
#         total_train_loss = 0
#         for inputs, targets in train_loader:
#             inputs, targets = inputs.to(device), targets.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = rmsle_loss(outputs, targets)
#             loss.backward()
#             optimizer.step()

#             total_train_loss += loss.item()

#         avg_train_rmsle = total_train_loss / len(train_loader)
#         total_test_loss = 0
#         model.eval()
#         with torch.no_grad():
#             for inputs, targets in test_loader:
#                 inputs, targets = inputs.to(device), targets.to(device)
#                 outputs = model(inputs)
#                 test_loss = rmsle_loss(outputs, targets)
#                 total_test_loss += test_loss.item()

#         avg_test_rmsle = total_test_loss / len(test_loader)
#         scheduler.step(avg_test_rmsle)

#         print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle}')

#     print(f'Best Test RMSLE so far: {scheduler.best}')


In [14]:
num_features = X_train.shape[1]


In [15]:
class PricePredictionModel(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(PricePredictionModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.2)
        
        self.layer2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.2)
        
        self.layer3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.2)
        
        self.layer4 = nn.Linear(64, 32)
        self.bn4 = nn.BatchNorm1d(32)
        self.dropout4 = nn.Dropout(0.1)
        
        self.output_layer = nn.Linear(32, output_size)

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.layer3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.bn4(self.layer4(x)))
        x = self.dropout4(x)
        x = self.output_layer(x)
        return x


In [13]:
model = PricePredictionModel(num_features).to(device)

# Use the training function already provided
train_model(model, train_loader, test_loader, num_epochs=10)

#Epoch 5: Train RMSLE = 0.6883012584179728, Test RMSLE = 0.6835928453002499
# Best Test RMSLE so far: 0.6832752976677257


Epoch 1: Train RMSLE = 0.7103170956413825, Test RMSLE = 0.6860611839599665
Epoch 2: Train RMSLE = 0.6905549636505591, Test RMSLE = 0.6841801157286848
Epoch 3: Train RMSLE = 0.6894629118511327, Test RMSLE = 0.683762296058531
Epoch 4: Train RMSLE = 0.6887406774495348, Test RMSLE = 0.6837902820019699
Epoch 5: Train RMSLE = 0.6881720342230563, Test RMSLE = 0.6835961619336686
Epoch 6: Train RMSLE = 0.6878651257972417, Test RMSLE = 0.6838404884550311
Epoch 7: Train RMSLE = 0.6876620969280219, Test RMSLE = 0.6838252511346595
Epoch 8: Train RMSLE = 0.6873545684333386, Test RMSLE = 0.6835608863926338
Epoch 9: Train RMSLE = 0.6868888843362309, Test RMSLE = 0.6832752976677257
Epoch 10: Train RMSLE = 0.6866839122186246, Test RMSLE = 0.6832874632584035
Best Test RMSLE so far: 0.6832752976677257


In [16]:
class EnhancedPricePredictionModel(nn.Module):
    def __init__(self, input_size, output_size=1):
        super(EnhancedPricePredictionModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.2)
        
        self.layer2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.1)
        
        self.layer3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(0.1)
        
        self.layer4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.dropout4 = nn.Dropout(0.1)
        
        self.output_layer = nn.Linear(64, output_size)

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.layer2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.bn3(self.layer3(x)))
        x = self.dropout3(x)
        x = torch.relu(self.bn4(self.layer4(x)))
        x = self.dropout4(x)
        x = self.output_layer(x)
        return x


In [19]:
model = EnhancedPricePredictionModel(num_features).to(device)

# Use the training function already provided
train_model(model, train_loader, test_loader, num_epochs=50)
# Epoch 1: Train RMSLE = 0.701097307975991, Test RMSLE = 0.6841780046707775
# Epoch 2: Train RMSLE = 0.6888048825449483, Test RMSLE = 0.6840163404823544
# Epoch 3: Train RMSLE = 0.6878524877785998, Test RMSLE = 0.683792892498503
# Epoch 4: Train RMSLE = 0.6872429579799324, Test RMSLE = 0.683677059799721
# Epoch 5: Train RMSLE = 0.6869078011756413, Test RMSLE = 0.6837233345654394
# Epoch 6: Train RMSLE = 0.6863807070894459, Test RMSLE = 0.6837847139004105

Epoch 1: Train RMSLE = 0.7539822288183858, Test RMSLE = 0.6851938397268962 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.6899419926804423, Test RMSLE = 0.6844406957920247 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.6882493528543755, Test RMSLE = 0.6845732714257409 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6874938286536473, Test RMSLE = 0.6836005338772091 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6868805261900853, Test RMSLE = 0.6843336063624458 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.686066268062078, Test RMSLE = 0.683395181901614 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.685587858133876, Test RMSLE = 0.6836968424531124 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6851818325337489, Test RMSLE = 0.6835474956886226 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.6848142597793547, Test RMSLE = 0.683199419487466 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6845869874318821, Test RMS

In [20]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.astype(np.float32)).unsqueeze(1)  # Add sequence dimension
X_test_tensor = torch.tensor(X_test.astype(np.float32)).unsqueeze(1)  # Add sequence dimension
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).view(-1, 1)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [21]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        out, _ = self.rnn(x)  # out shape: (batch_size, sequence_length, hidden_dim)
        out = out[:, -1, :]  # get the last sequence output
        out = self.fc(out)
        return out


In [22]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        out, (hn, cn) = self.lstm(x)  # out shape: (batch_size, sequence_length, hidden_dim)
        out = out[:, -1, :]  # get the last sequence output
        out = self.fc(out)
        return out


In [23]:
model = LSTMModel(input_dim=num_features, hidden_dim=100, output_dim=1).to(device)  # Adjust dimensions as necessary
train_model(model, train_loader, test_loader, num_epochs=10)
# Best Test RMSLE so far: 0.6883052102114385

# Minimum Train RMSLE so far: 0.6873640940228356
# Minimum Test RMSLE so far: 0.6873739421027812


Epoch 1: Train RMSLE = 0.9505505237734364, Test RMSLE = 0.697044079488693 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.6932499246288626, Test RMSLE = 0.6904735023100457 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.6900384213120617, Test RMSLE = 0.6890186226586268 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6887344956531883, Test RMSLE = 0.6880314034546913 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6882432286220607, Test RMSLE = 0.6881312627368494 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6879552400327036, Test RMSLE = 0.6877571556359303 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.6875714813525351, Test RMSLE = 0.6875845276555607 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6872967017775471, Test RMSLE = 0.6873673786407268 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.6869023333384512, Test RMSLE = 0.6874346182975756 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6867546385123663, Test 

In [24]:
model = RNNModel(input_dim=num_features, hidden_dim=10, output_dim=1).to(device)  # Adjust dimensions as necessary
train_model(model, train_loader, test_loader, num_epochs=10)
#Best Test RMSLE so far: 0.9206008657493835

# Minimum Train RMSLE so far: 0.6880693981135202
# Minimum Test RMSLE so far: 0.6876124085880347

Epoch 1: Train RMSLE = 0.8828789826205348, Test RMSLE = 0.6956600377435262 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.6926597734233993, Test RMSLE = 0.6904328210895937 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.690628907961852, Test RMSLE = 0.690253002943668 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6901779458925653, Test RMSLE = 0.689415318493647 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6900961761549715, Test RMSLE = 0.6895427988699245 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6896527157287613, Test RMSLE = 0.6887947437768569 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.6892808137731435, Test RMSLE = 0.6886071407885574 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6889113583781153, Test RMSLE = 0.6881322431807363 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.688569586635652, Test RMSLE = 0.6878116669206679 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6881853451214747, Test RMS

In [25]:
import torch
import torch.nn as nn

class RBNFModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(RBNFModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        out, (hn, cn) = self.lstm(x)  # out shape: (batch_size, sequence_length, hidden_dim)
        out = out[:, -1, :]  # we only use the output of the last time step
        out = self.batch_norm(out)
        out = self.fc(out)
        return out


In [27]:
model = RBNFModel(input_dim=num_features, hidden_dim=10, output_dim=1).to(device)  # Adjust dimensions as necessary
train_model(model, train_loader, test_loader, num_epochs=30)

# Minimum Train RMSLE so far: 0.6887654505139725
# Minimum Test RMSLE so far: 0.6873131738441437

Epoch 1: Train RMSLE = 0.8586716178261418, Test RMSLE = 0.6924337216838937 , Current LR = [0.02000080000000002]
Epoch 2: Train RMSLE = 0.6927101926093303, Test RMSLE = 0.6900119445625881 , Current LR = [0.0400006]
Epoch 3: Train RMSLE = 0.6909530170225817, Test RMSLE = 0.6890279623707493 , Current LR = [0.06000040000000001]
Epoch 4: Train RMSLE = 0.6905018127347654, Test RMSLE = 0.688458244570511 , Current LR = [0.0800002]
Epoch 5: Train RMSLE = 0.6903444401760224, Test RMSLE = 0.688870332512101 , Current LR = [0.1]
Epoch 6: Train RMSLE = 0.6899808727512022, Test RMSLE = 0.6878011852461631 , Current LR = [0.0800002]
Epoch 7: Train RMSLE = 0.6896246426070074, Test RMSLE = 0.6877104775682761 , Current LR = [0.06000040000000001]
Epoch 8: Train RMSLE = 0.6891783315141721, Test RMSLE = 0.6877810799187846 , Current LR = [0.0400006]
Epoch 9: Train RMSLE = 0.6889893982908716, Test RMSLE = 0.6873621200343288 , Current LR = [0.02000080000000002]
Epoch 10: Train RMSLE = 0.6887345827619922, Test R

In [50]:
!pip install pytorch-tabnet


   ---------------------------------------- 0.0/44.5 kB ? eta -:--:--
   ---------------------------------------- 44.5/44.5 kB 2.1 MB/s eta 0:00:00


In [4]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from pytorch_tabnet.callbacks import Callback

X = train.drop(columns=['price'])
y = np.log1p((train['price'])).values.reshape(-1, 1) # talk about this 

# Initialize TabNetRegressor
model = TabNetRegressor(device_name='cuda' if torch.cuda.is_available() else 'cpu')

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)



c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\461p\ECE-461P-Term-Project\.venv\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [6]:

model.fit(
  X_train, y_train,
  eval_set=[(X_test, y_test)],
  eval_name=['eval'],
  eval_metric=['rmse'],
  max_epochs=100,
  patience=50,  # Early stopping based on validation loss
  batch_size=1024,  # Adjust according to your GPU memory
  virtual_batch_size=128,  # Mini batch size for "Ghost Batch Normalization"
  num_workers=0,
  drop_last=False
)
# Predict and evaluate
predictions = model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, predictions))
print(f"Test RMSE: {rmse}")

#Stop training because you reached max_epochs = 100 with best_epoch = 86 and best_eval_rmse = 0.69104 Test RMSE: 0.6910353193627529


epoch 0  | loss: 0.54274 | eval_rmse: 0.69856 |  0:01:17s
epoch 1  | loss: 0.48695 | eval_rmse: 0.69605 |  0:02:16s
epoch 2  | loss: 0.48567 | eval_rmse: 0.69344 |  0:03:10s
epoch 3  | loss: 0.4844  | eval_rmse: 0.69511 |  0:04:02s
epoch 4  | loss: 0.48378 | eval_rmse: 0.69366 |  0:05:17s
epoch 5  | loss: 0.48347 | eval_rmse: 0.69348 |  0:06:35s
epoch 6  | loss: 0.48357 | eval_rmse: 0.69595 |  0:07:52s
epoch 7  | loss: 0.48325 | eval_rmse: 0.69243 |  0:08:37s
epoch 8  | loss: 0.4834  | eval_rmse: 0.69331 |  0:09:24s
epoch 9  | loss: 0.48294 | eval_rmse: 0.69339 |  0:10:09s
epoch 10 | loss: 0.48313 | eval_rmse: 0.69269 |  0:10:56s
epoch 11 | loss: 0.48443 | eval_rmse: 0.69336 |  0:11:42s
epoch 12 | loss: 0.48329 | eval_rmse: 0.69556 |  0:12:27s
epoch 13 | loss: 0.48311 | eval_rmse: 0.69327 |  0:13:13s
epoch 14 | loss: 0.4829  | eval_rmse: 0.69274 |  0:13:58s
epoch 15 | loss: 0.48234 | eval_rmse: 0.69239 |  0:14:44s
epoch 16 | loss: 0.48236 | eval_rmse: 0.69248 |  0:15:32s
epoch 17 | los

c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\461p\ECE-461P-Term-Project\.venv\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Test RMSE: 0.6910353193627529
